In [4]:
import requests
import json
from bs4 import BeautifulSoup
from typing import List
import time
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_urls_lists(search_page_url: str) -> List[str]:
    response = requests.get(search_page_url, params = {})
    soup = BeautifulSoup(response.text, 'html.parser')

    list_of_urls=[]

    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            list_of_urls.append(link.get('href'))
    return list_of_urls

def extract_x_urls(number_of_urls: int=30) -> List[str]:
    immo_base_url = "https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page="
    immo_end_url = "&orderBy=relevance"
    searchpages_urls_list =[]

    for index in range(1,round(min(334,number_of_urls/30 + 1))):
        full_url = immo_base_url + str(index) + immo_end_url
        searchpages_urls_list.append(full_url)
    
    full_urls_list =[]

    with ThreadPoolExecutor() as pool:
        full_urls_list=list(pool.map(extract_urls_lists, searchpages_urls_list))  

    if number_of_urls>9990:
        immo_base_url = "https://www.immoweb.be/fr/recherche/appartement/a-vendre?countries=BE&page="
        immo_end_url = "&orderBy=relevance"
        searchpages_urls_list =[]

        for index in range(1,round(min(334,(number_of_urls-9990)/30 + 1))):
            full_url = immo_base_url + str(index) + immo_end_url
            searchpages_urls_list.append(full_url)

        with ThreadPoolExecutor() as pool:
            full_urls_list+=list(pool.map(extract_urls_lists, searchpages_urls_list))  
            
    return list(set(sum(full_urls_list,[])))
extract_x_urls(3000)


['https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10565436',
 'https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10562026',
 'https://www.immoweb.be/en/classified/house/for-sale/gent/9000/10567419',
 'https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/zwevegem/8550/10567607',
 'https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10567677',
 'https://www.immoweb.be/en/classified/house/for-sale/watermael-boitsfort/1170/10567511',
 'https://www.immoweb.be/en/classified/duplex/for-sale/brussels/1000/10567500',
 'https://www.immoweb.be/en/classified/duplex/for-sale/molenbeek-saint-jean/1080/10567505',
 'https://www.immoweb.be/en/classified/apartment/for-sale/saint-ghislain/7333/10566161',
 'https://www.immoweb.be/en/classified/apartment/for-sale/saint-ghislain/7333/10566160',
 'https://www.immoweb.be/en/classified/apartment/for-sale/saint-ghislain/7333/10566159',
 'https://www.immoweb.be/en/classified/new-real-

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
pd.set_option('display.width', None)
def make_one_data_frame(url):
  """
  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)
  """
  try:
    response = requests.get(url)
    test_page = response.text

    dfs = pd.read_html(test_page)

    full_df = pd.concat(dfs).dropna(thresh=2).T
    full_df.columns = full_df.iloc[0]
    full_df = full_df[1:]
    full_df = full_df.loc[:, full_df.columns.isin(['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace'])]

    return full_df
  except:
    return ['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace']
make_one_data_frame("https://www.immoweb.be/en/classified/house/for-sale/kortrijk/8500/10564478")

#data.to_csv("datanew.csv")
  # pd.read_html reads in all tables and returns a list of DataFrames



,Number of frontages,Living area,Bedrooms,Bathrooms,Price
1,2,112 m² square meters,2,1,"€ 190,000 190000 €"


In [5]:

list_of_urls = extract_x_urls(10000)
print(len(list_of_urls))
print('list ready')

dataframes_list = []
with ThreadPoolExecutor() as pool:
    results = list(pool.map(make_one_data_frame, list_of_urls))
    for result in results:
        dataframes_list.append(result)

full_df = pd.concat(dataframes_list).reset_index(drop=True)
full_df.dropna(thresh=2,inplace=True)

print('done extracting')

#full_df.to_csv("df.csv")
full_df

#full_df.loc[:, full_df.columns.isin(['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace'])]


9990
list ready


ValueError: No tables found